## 1. Import Dependencies

In [48]:
import keras
import os
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import json
import numpy as np
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import *
import webbrowser
import requests
# from pycricbuzz import Cricbuzz
# import billboard
import time

## 2. Constants and Paths

In [13]:
DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
MODEL_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'models'))
INTENTS_PATH = os.path.join(DATA_PATH, 'intents.json')
WORDS_PATH = os.path.join(DATA_PATH, 'words.pkl')
CLASSES_PATH = os.path.join(DATA_PATH, 'classes.pkl')

## 3. Tokenize and Lemmatize the Intents

In [3]:
# Initializing the WordNet lemmatizer
lemmatizer=WordNetLemmatizer()


In [4]:
# Lists to store words, classes, and documents
words=[]
classes=[]
documents=[]

# Ignored characters in tokenization
ignore=['?','!',',',"'s"]

In [5]:
# Reading data from intents.json file
data_file=open(INTENTS_PATH).read()
intents=json.loads(data_file)

In [6]:
# Loop through each intent in the intents dictionary
for intent in intents['intents']:
    # Loop through each pattern in the current intent
    for pattern in intent['patterns']:
        # Tokenize the pattern into words
        w = nltk.word_tokenize(pattern)

        # Extend the words list with the tokenized words
        words.extend(w)

        # Append a tuple containing (tokenized words, intent tag) to the documents list
        documents.append((w, intent['tag']))

        # Add intent tag to classes list, if still missing
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# Lemmatize and lowercase each word in the words list, excluding ignored characters
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]

# Remove duplicate words, sort, and convert to a list
words = sorted(list(set(words)))

# Sort and convert classes to a list
classes = sorted(list(set(classes)))

# Save the preprocessed words and classes to pickle files for later use
pickle.dump(words,open(WORDS_PATH,'wb'))
pickle.dump(classes,open(CLASSES_PATH,'wb'))

## 4. Preparing Training Data

In [8]:
# List to store training data and initialize an empty output
training = []
output_empty = [0]*len(classes)

In [9]:
# Iterate through each document in the documents list
for doc in documents:
    # Initialize an empty bag of words for the current document
    bag = []

    # Get the tokenized pattern from the document
    pattern = doc[0]

    # Lemmatize and lowercase each word in the pattern
    pattern = [lemmatizer.lemmatize(word.lower()) for word in pattern ]

    # Iterate through each word in the list of all words
    for word in words:
        # Check if the word is in the pattern, append 1 if true, 0 if false
        if word in pattern:
            bag.append(1)
        else:
            bag.append(0)

    output_row = list(output_empty)
    # Set the value at the index corresponding to the class of the current document to 1
    output_row[classes.index(doc[1])] = 1

    # Append a pair of the bag of words and the output row to the training list
    training.append((bag, output_row))

In [10]:
# Shuffle the training data randomly
random.shuffle(training)

# Convert the training list to a NumPy array
# training = np.array(training)

# Separate the features (X_train) and labels (y_train) from the training array
# X_train = list(training[:,0])
# y_train = list(training[:,1])
X_train = np.array([item[0] for item in training])  # Convert bag to NumPy array
y_train = np.array([item[1] for item in training])  # Convert output_row to NumPy array

## 5. Modelling

In [33]:
def initialize_model():
    model=Sequential()
    model.add(Dense(128, activation='relu', input_shape=(len(X_train[0]),)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(y_train[0]), activation='softmax'))

    adam = keras.optimizers.Adam(0.001)
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)

    return model, early_stopping

In [35]:
model, early_stopping = initialize_model()


In [36]:
def train_model(model, X_train, y_train, early_stopping):
    history = model.fit(X_train, y_train,
                        epochs=200,
                        batch_size=10,
                        callbacks=[early_stopping],
                        verbose=-1)
    
    return history

In [45]:
history = train_model(model, X_train, y_train, early_stopping)
print(history.history.keys())
max(history.history['accuracy'])

Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
dict_keys(['loss', 'accuracy'])


0.9642857313156128

In [46]:
def save_model(history):
    for version in range(1, 99):
        file = f'kaybot_model_{str(version)}.h5'
        file_path = os.path.join(MODEL_PATH, file)
        if not os.path.exists(file_path):
            model.save(file_path, history)
            print(f'✅ Model saved as "{file}"')
            break
    return None


In [47]:
save_model(history)

✅ Model saved as "kaybot_model_1.h5"


/home/jarisfenner/.pyenv/versions/3.10.6/envs/KayBot/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 6. 

In [ ]:
model = load_model('mymodel.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))